In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import json
from threading import Thread,Lock,Event

In [2]:
headers = {
    'authority': 'www.bigbasket.com',
    'method': 'GET',
    'path': '/listing-svc/v2/products?type=pc&slug=foodgrains-oil-masala&page=1',
    'scheme': 'https',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Cookie': 'x-entry-context-id=100; x-entry-context=bb-b2c; _bb_locSrc=default; x-channel=web; _bb_loid=j:null; _bb_bhid=; _bb_nhid=1723; _bb_vid=MTI4MjA0NDYxMzQ=; _bb_dsevid=; _bb_dsid=; _bb_cid=1; _bb_aid=MzA4NTgxODk5Nw==; csrftoken=HhsvZkeZFNA7XdmHV5IIUT49QLIvvrXbMF4gCJoPWxShyovvBqwpje2fFCm84nEj; _bb_home_cache=4cca0b57.1.visitor; bb2_enabled=true; ufi=1; bigbasket.com=2f61e0d4-9d4f-4530-ba94-334049c46704; _gcl_au=1.1.2033074425.1718993968; _ga=GA1.1.1843003858.1718993968; jarvis-id=5d0d66fb-63a4-4685-a900-2dfc7b925de7; _ga_FRRYG5VKHX=GS1.1.1719002693.3.1.1719002693.60.0.0; csurftoken=egUF8w.MTI4MjA0NDYxMzQ=.1720255452916.XQjr+mZO4SgN4wjdV3jFFPEr7mvcgjkbbVDcgrCBe/w=',
    'Sec-Ch-Ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Microsoft Edge";v="126"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0',
    'Content-Type': 'application/json',
    'Newrelic': 'eyJ2IjpbMCwxXSwiZCI6eyJ0eSI6IkJyb3dzZXIiLCJhYyI6IjgzNzMwNCIsImFwIjoiMTgzNDk4NzAwMiIsImlkIjoiYmM1NzlmNzRiNjA1ZjkzYyIsInRyIjoiZWIzNTliYTAzZjg3NGFmNTA4MGI3YWY5NjRkN2QzNjIiLCJ0aSI6MTcyMDIyOTg3MTQyNH19',
    'Priority': 'u=1, i'
}

In [69]:
product_pages = []
lock = Lock()
flag=True
def scrap_page(page,lock):
    global flag
    url = f"https://www.bigbasket.com/listing-svc/v2/products?type=pc&slug=fruits-vegetables&page={page}"   #add category after slug=
    try:
        response = requests.get(url, headers=headers)
        data = response.json()
        with lock:
            try:
                product_pages.append(data["tabs"][0]["product_info"].get("products",[]))
            except Exception as e:
                print(f"Error processing page {page}: {e}")
    except Exception as e:
        print(f"Request failed for page {page}: {e}")
        flag=False

if __name__ == "__main__":
    all_threads = []
    number_threads = 2
    page =1
    while True:
        try:
            for _ in range(number_threads):
                t = Thread(target=scrap_page, args=(page,lock))
                if not flag :
                    raise Exception("Page Limit Reached")
                all_threads.append(t)
                t.start()
                # print(page)
                page += 1
                
            for t in all_threads:
                t.join()
            
            all_threads = []
        except Exception as e:
            print(f"Total pages scraped: {len(product_pages)}")
            # print(e)
            # print(product_info)
            break

Request failed for page 42: Expecting value: line 1 column 1 (char 0)
Total pages scraped: 41


In [70]:
product_detail=[]
for page in product_pages:
    for product in page:
        id=product.get("id")
        if not id:
            continue

        Ean_code= product.get("ean_code")
        if not Ean_code or Ean_code=="0":
            continue

        price=product["pricing"]["discount"].get("mrp")
        if not price:
            continue
            
        Image_link=product.get("images")
        if not Image_link:
            continue
            
        Brand=product["brand"].get("name")
        if not Brand:
            continue
            
        title=product.get("desc")
        if not title:
            continue
            
        # availability=product["availability"].get("avail_status")
        # if not availability:
        #     availability="Unavailable"
            
        tlc,mlc,llc=product["category"].get("tlc_name"), product["category"].get("mlc_name"), product["category"].get("llc_name")
        category_tlc,category_mlc,category_llc=(tlc,mlc,llc)
        
        mag=product.get("magnitude")
        if not mag:
            continue
            
        unit=product.get("unit")
        if not unit:
            continue

        children=product.get("children") if product.get("children") else 'NAN'

        w=product.get("w")
        if not w:
            continue
        l={"x","X"}
        count=1
        try:
            v="".join(list(l.intersection(w)))
            temp=w.split(v)
            check=re.findall("[A-Za-z ]",temp[0].strip())
            if len(temp)==2 and not check:
                count=int("".join(re.findall("[0-9]",temp[0])))
                w_mag_str="".join((re.findall("[0-9.+]",temp[1])))
                if w_mag_str:
                    try:
                        w_mag = float(w_mag_str)
                    except ValueError:
                        w_mag = w_mag_str
                else:
                    w_mag = None 
                w_unit="".join((re.findall("[A-Za-z/ ]",temp[1]))).strip()
            else:
                raise Exception("not found")
        except:
            w_mag="".join((re.findall("[0-9.+Xx]",w)))
            w_unit="".join((re.findall("[A-Za-z+/ ]",w))).strip()
 
        if not w_unit or not w_mag:
            title=f"({title}){Brand}[{mag}]{{{unit}}}<{price}>"
        else:    
            title=f"({title}){Brand}[{count}x{w_mag}]{{{w_unit}}}<{price}>"
                  
        product_detail.append({"Id":id,"EAN code":f'"{Ean_code}"',"Title":title,"Brand":Brand,"Magnitude":mag,"Unit":unit,"Count":count,"w_mag":w_mag,"w_unit":w_unit,"Price":price,"category_tlc":category_tlc,"category_mlc":category_mlc,"category_llc":category_llc,"Children":children,"Image":Image_link})
        
print("done")
# print(product_detail)


done


In [73]:
keys=product_detail[0].keys()

#change mode from w to a as per requirement
with open("scraped_data_fruits-vegetables_Raw.csv","w", newline='', encoding='utf-8') as output_file:
    dict_writer = csv.DictWriter(output_file, fieldnames=keys)
    dict_writer.writeheader()  # Uncomment if writing the header for the first time
    dict_writer.writerows(product_detail)

In [74]:
import pandas as pd
df=pd.read_csv("scraped_data_fruits-vegetables_Raw.csv")
df

,Id,EAN code,Title,Brand,Magnitude,Unit,Count,w_mag,w_unit,Price,category_tlc,category_mlc,category_llc,Children,Image
0,40217762,"""84021776200296""",(Cherry)fresho![1x250]{g}<291.78>,fresho!,250,g,1,250,g,291.78,Fruits & Vegetables,Fresh Fruits,Seasonal Fruits,NAN,[{'s': 'https://www.bigbasket.com/media/upload...
1,10000111,"""10000111""","(Methi/Venthaya Keerai - Cleaned, without root...",fresho!,1000,g,1,1,kg,223.29,Fruits & Vegetables,Fresh Vegetables,Leafy Vegetables,"[{'id': '10000113', 'desc': 'Methi/Venthaya Ke...",[{'s': 'https://www.bigbasket.com/media/upload...
2,10000369,"""10000369""",(Guava (Loose))fresho![1x1]{kg}<116.44>,fresho!,1000,g,1,1,kg,116.44,Fruits & Vegetables,Fresh Fruits,"Banana, Sapota & Papaya","[{'id': '10000370', 'desc': 'Guava (Loose)', '...",[{'s': 'https://www.bigbasket.com/media/upload...
3,10000164,"""1000016""",(Radish - White (Loose))fresho![1x1]{kg}<64.38>,fresho!,1000,g,1,1,kg,64.38,Fruits & Vegetables,Fresh Vegetables,Root Vegetables,"[{'id': '10000165', 'desc': 'Radish - White (L...",[{'s': 'https://www.bigbasket.com/media/upload...
4,20003955,"""20003955""",(Mini Orange Imported)fresho![1x1]{kg}<365.75>,fresho!,1000,g,1,1,kg,365.75,Fruits & Vegetables,Fresh Fruits,"Kiwi, Melon, Citrus Fruit","[{'id': '20003956', 'desc': 'Mini Orange Impor...",[{'s': 'https://www.bigbasket.com/media/upload...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1842,40104708,"""40104708""",(Orange - Nagpur Premium)fresho![1x4]{pcs}<65>,fresho!,4,pc,1,4,pcs,65.00,Fruits & Vegetables,Fresh Fruits,"Kiwi, Melon, Citrus Fruit",NAN,[{'s': 'https://www.bigbasket.com/media/upload...
1843,40084209,"""40084209""","(Bitter Gourd Economy, Institutional)fresho![1...",fresho!,1000,g,1,1,kg,34.00,Fruits & Vegetables,Fresh Vegetables,"Gourd, Pumpkin, Drumstick",NAN,[{'s': 'https://www.bigbasket.com/media/upload...
1844,40084197,"""40084197""","(Banana Robusta - Direct, Institutional)fresho...",fresho!,1000,g,1,1,kg,39.00,Fruits & Vegetables,Fresh Fruits,"Banana, Sapota & Papaya",NAN,[{'s': 'https://www.bigbasket.com/media/upload...
1845,20001094,"""20001094""",(Chinna Rasalu Mango)fresho![1x1]{kg}<202.74>,fresho!,1000,g,1,1,kg,202.74,Fruits & Vegetables,Fresh Fruits,Mangoes,NAN,[{'s': 'https://www.bigbasket.com/media/upload...


In [ ]:
#make a file directory and save all csv file

In [75]:
import pandas as pd
import os

# Load the CSV file into a DataFrame
df = pd.read_csv('fruits-vegetables\scraped_data_fruits-vegetables_TLC.csv')

# Group the DataFrame by 'category_mlc'
grouped = df.groupby('category_mlc')

# Create a directory to save the grouped CSV files
output_dir = 'fruits-vegetables\MLC_grouped_csv_files'
os.makedirs(output_dir, exist_ok=True)

# Iterate over each group and save each group to a separate CSV file
for name, group in grouped:
    # Ensure the filename is valid and not too long
    valid_filename = name.replace('/', '_').replace('\\', '_')
    output_path = os.path.join(output_dir, f'{valid_filename}.csv')
    
    # Save the group to a CSV file
    group.to_csv(output_path, index=False)

print(f"Grouped data successfully saved to CSV files in the '{output_dir}' directory.")


Grouped data successfully saved to CSV files in the 'fruits-vegetables\MLC_grouped_csv_files' directory.


In [76]:
import pandas as pd
import os

# Load the CSV file into a DataFrame
df = pd.read_csv('fruits-vegetables\scraped_data_fruits-vegetables_TLC.csv')

# Group the DataFrame by 'category_mlc'
grouped = df.groupby('category_llc')

# Create a directory to save the grouped CSV files
output_dir = 'fruits-vegetables\LLC_grouped_csv_files'
os.makedirs(output_dir, exist_ok=True)

# Iterate over each group and save each group to a separate CSV file
for name, group in grouped:
    # Ensure the filename is valid and not too long
    valid_filename = name.replace('/', '_').replace('\\', '_')
    output_path = os.path.join(output_dir, f'{valid_filename}.csv')
    
    # Save the group to a CSV file
    group.to_csv(output_path, index=False)

print(f"Grouped data successfully saved to CSV files in the '{output_dir}' directory.")


Grouped data successfully saved to CSV files in the 'fruits-vegetables\LLC_grouped_csv_files' directory.
